<a href="https://colab.research.google.com/github/CNG85-HW/CNG_Voodeo/blob/main/CNG_Voodeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image


# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the model path in Google Drive
model_dir = "/content/drive/My Drive/models/ssd_mobilenet_v2_coco_2018_03_29"
model_path = os.path.join(model_dir, "saved_model")

# Step 3: Check if the model directory exists; if not, prompt user to upload it
if not os.path.exists(model_path):
    print(f"Model directory not found at {model_path}. Please upload the model to the folder:\n{model_dir}")
    raise FileNotFoundError("Model not found. Upload it to Google Drive first.")

# Step 4: Load the TensorFlow model
try:
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", str(e))
    raise

# Function to upload multiple images and one video
def upload_files():
    uploaded = files.upload()  # Drag and drop files here
    reference_images = {}
    video_path = None

    for file_name in uploaded.keys():
        if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            object_name = file_name.rsplit('.', 1)[0]  # Use file name (without extension) as the label
            reference_images[object_name] = file_name
        elif file_name.lower().endswith(('.mp4', '.avi')):
            video_path = file_name

    if not reference_images or not video_path:
        raise ValueError("Please upload at least one reference image and one video file.")
    return reference_images, video_path

# Function to preprocess reference images
def preprocess_images(image_paths):
    processed_images = {}
    for label, path in image_paths.items():
        image = Image.open(path)
        image_np = np.array(image)
        input_tensor = tf.convert_to_tensor(image_np)
        processed_images[label] = input_tensor[tf.newaxis, ...]
    return processed_images

# Function to detect objects in the video
def detect_objects_in_video(video_path, reference_images):
    processed_images = preprocess_images(reference_images)

    cap = cv2.VideoCapture(video_path)
    frame_number = 0
    detection_results = {label: [] for label in processed_images.keys()}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        input_tensor = tf.convert_to_tensor(frame)
        input_tensor = input_tensor[tf.newaxis, ...]

        detections = model.signatures['serving_default'](input_tensor)
        detection_scores = detections['detection_scores'].numpy()
        detection_boxes = detections['detection_boxes'].numpy()
        detection_classes = detections['detection_classes'].numpy()

        for label, reference_tensor in processed_images.items():
            if detection_scores[0][0] > 0.85:  # Adjust confidence threshold as needed
                print(f"Object '{label}' detected in frame {frame_number} with confidence {detection_scores[0][0]:.2f}")
                detection_results[label].append(frame_number)
                cv2_imshow(frame)

        frame_number += 1

    cap.release()

    for label, frames in detection_results.items():
        print(f"\nObject '{label}' detected in {len(frames)} frame(s): {frames}")

# Main function to run the script
def main():
    print("Upload your reference images and video.")
    reference_images, video_path = upload_files()
    detect_objects_in_video(video_path, reference_images)

# Run the main function
if __name__ == "__main__":
    main()
